In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 3*3의 정사각 컨볼루션행렬
        # 컨벌루션 커널 정의
        self.conv1 = nn.Conv2d(1,6,3) # in_channels 1, out_channels 6, kernel_size 3
        self.conv2 = nn.Conv2d(6,16,3) # in_channels 6, out_channels 16, kernel_size 3
        # affine 연산
        # torch.nn.linear(a,b, bias = bool): 입력이 a개의 features, 출력이 b개의 features인 y = x*A.T + b 를 만듦.
        self.fc1 = nn.Linear(16*6*6, 120) # 6*6은 이미지 차원?
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))# convolution -> relu -> max_pool
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 뭔가 linearize 해서 결국 길이 10짜리로 만드나?
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0092,  0.0187,  0.0246, -0.0054,  0.0225,  0.0322, -0.0087, -0.1062,
         -0.0006, -0.0031]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad() #모든 grad를 0으로 설정
out.backward(torch.randn(1,10))

# 손실함수 (Loss Function) 

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print('target is {}'.format(target))

tensor(0.8398, grad_fn=<MseLossBackward>)
target is tensor([[ 0.1626,  2.3320, -0.7282, -0.4214,  1.0069, -0.7885,  0.7164, -0.3183,
          0.2647, -0.0057]])


In [10]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [14]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0045, -0.0047,  0.0100,  0.0106, -0.0044, -0.0074])


In [15]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [16]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()